# Lab 6: The *Hubel-Wiesel* model of cortical networks

Advanced Topics in Machine Learning -- Spring 2023, UniTS

<a target="_blank" href="https://colab.research.google.com/github/ganselmif/adv-ml-units/blob/main/solutions/AdvML_UniTS_2023_Lab_06_Cortex_Hubel_Wiesel_Solved.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

#### Overview of *Hubel & Wiesel* model

In this lab, we will attempt an implementation of (a *machine-learning-flavoured* version of) the *Hubel & Wiesel* model of cortical networks. The model is a simplified (and, today we know, not entirely realistic) model of the primary visual cortex of the mammalian brain.

According to the model, the cortex is composed by sets of *simple cells* and *complex cells*.

- *Simple cells* are sensitive to a specific orientation of the edges in the visual field.
- *Complex cells* integrate the signal received by a set of *simple cells*.

By abstracting away the anatomical details, we can write:

- For *simple cells*: $s_{ij}(x)=x^{T}(g_{j}(w_{i}))$;
- For *complex cells*: $c_{h,i}(x)=\sum_{j}\sigma_{h}(s_{ij}(x))$

where $x$ is an input vector, $w_{i}$ is the weight vector of the $i$-th simple cell, $g_{j}$ is a specific realization of the $g$ transformation, $\sigma_{h}$ is the activation function of the $h$-th complex cell, and $s_{ij}$ and $c_{h,i}$ are the output of the $i$-th simple cell and the $h$-th complex cell, respectively.

From the definitions above, we expect:
- *Simple cells* to be equivariant to transformations of the inputs, i.e. $s_{i}(gx)=P_{g}s_{i}(x)$;
- *Complex cells* to be invariant to transformations of the inputs, i.e. $c_{h,i}(x) = c_{h,i}(gx)$,

where $P_{g}$ is an (any!) element of the group generating the transformation $g$.

#### Lab to-do:

1. Implement a *simple* cell of the *Hubel & Wiesel* model, using central (vertical) flip as the transformation $g$ of interest.
Note that such transformation is associated to the unitary group generated by $g_1 = \text{Identity}$ and $g_2 = \text{Flip}$, and as such requires (and admits) only two specific transformations of the data to be implemented.

In [1]:
from typing import Callable
import numpy as np

In [2]:
def simplecell(x, g_i):
    if isinstance(g_i, Callable):
        return g_i(x)
    else:
        return np.matmul(g_i, x)

In [3]:
def flip_simplecells(x):
    return simplecell(x, np.eye(x.shape[0])), simplecell(x, np.fliplr)

2. Implement a *complex* cell of the *Hubel & Wiesel* model.

In [4]:
def complexcell(x_iterable):
    return 1 / (1 + np.exp(-np.sum(np.array(x_iterable), axis=0)))

3. Verify the expected properties of the *simple* and *complex* cells in terms of equivariance and invariance with respect to the same transformation $g$ described above, on synthetic data (you can generate some yourself!).


In [5]:
# Generate some data
xsize = (28, 28)
nperms = 2

xlist = [
    np.random.rand(*xsize),
]
for _ in range(nperms - 1):
    xlist.append(np.fliplr(xlist[0]))

x = np.stack(xlist)

In [6]:
# Compute the outputs of the simple cells
ylist_sc = []
for x_i in x:
    ylist_sc.append(flip_simplecells(x_i))
y_sc = np.stack(ylist_sc)

In [7]:
# Compute the outputs of the complex cells
ylist_cc = []
for y_i in y_sc:
    ylist_cc.append(complexcell(y_i))
y_cc = np.stack(ylist_cc)

In [8]:
# Equivariance of simple cells
ysc_sorted = np.sort(y_sc.reshape(y_sc.shape[0], -1), axis=1)
if np.isclose(ysc_sorted[0], ysc_sorted).all():
    print("Success (equivariance)!")

# Invariance of complex cells
if np.isclose(
    y_cc.reshape(y_cc.shape[0], -1)[0], y_cc.reshape(y_cc.shape[0], -1)
).all():
    print("Success (invariance)!")


Success (equivariance)!
Success (invariance)!
